In [4]:
from keras.datasets import mnist 
import matplotlib.pyplot as plt 
from keras.optimizers import Adam 
from keras import utils 
import numpy as np 
import matplotlib.pyplot as plt 
from tensorflow.keras.models import load_model 
import cv2
dataset = mnist
(x_train, y_train), (x_test, y_test) = dataset.load_data() #translate the data to image 
x_train=[cv2.cvtColor(cv2.resize(i,(48,48)),cv2.COLOR_GRAY2BGR)for i in x_train]  #x_train shape is (10000,48,48,3)
x_train=np.concatenate([arr[np.newaxis]for arr in x_train]).astype('float32')
x_train=x_train/255
x_test=[cv2.cvtColor(cv2.resize(i,(48,48)),cv2.COLOR_GRAY2BGR)for i in x_test]
x_test=np.concatenate([arr[np.newaxis]for arr in x_test]).astype('float32')
x_test=x_test/255

lenet5 = load_model('Model/vgg_mnist.h5')#load lenet5 model trained with mnist dataset 
lenet5.compile(optimizer=Adam(),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [6]:
beta = 0.32200
sample(x_test,y_test,beta)

Beta is reduced by 0.0001, the new beta is 0.3219
Beta is reduced by 0.0001, the new beta is 0.32180000000000003
Beta is reduced by 0.0001, the new beta is 0.32170000000000004
Beta is reduced by 0.0001, the new beta is 0.32160000000000005
Beta is reduced by 0.0001, the new beta is 0.32150000000000006
Beta is reduced by 0.0001, the new beta is 0.3214000000000001
Beta is reduced by 0.0001, the new beta is 0.3213000000000001
Beta is reduced by 0.0001, the new beta is 0.3212000000000001
Beta is reduced by 0.0001, the new beta is 0.3211000000000001
Beta is reduced by 0.0001, the new beta is 0.3210000000000001
Beta is reduced by 0.0001, the new beta is 0.32090000000000013
Beta is reduced by 0.0001, the new beta is 0.32080000000000014
Beta is reduced by 0.0001, the new beta is 0.32070000000000015
Beta is reduced by 0.0001, the new beta is 0.32060000000000016
Beta is reduced by 0.0001, the new beta is 0.3205000000000002
Beta is reduced by 0.0001, the new beta is 0.3204000000000002
Beta is redu

Beta is reduced by 0.0001, the new beta is 0.24450000000000854
Beta is reduced by 0.0001, the new beta is 0.24440000000000855
Beta is reduced by 0.0001, the new beta is 0.24430000000000857
Beta is reduced by 0.0001, the new beta is 0.24420000000000858
Beta is reduced by 0.0001, the new beta is 0.2441000000000086
Beta is reduced by 0.0001, the new beta is 0.2440000000000086
Beta is reduced by 0.0001, the new beta is 0.2439000000000086
Beta is reduced by 0.0001, the new beta is 0.24380000000000862
Beta is reduced by 0.0001, the new beta is 0.24370000000000863
Beta is reduced by 0.0001, the new beta is 0.24360000000000864
Beta is reduced by 0.0001, the new beta is 0.24350000000000865
Beta is reduced by 0.0001, the new beta is 0.24340000000000866
Beta is reduced by 0.0001, the new beta is 0.24330000000000868
Beta is reduced by 0.0001, the new beta is 0.2432000000000087
Beta is reduced by 0.0001, the new beta is 0.2431000000000087
Beta is reduced by 0.0001, the new beta is 0.243000000000008

0.20200000000001322

In [5]:
def sample(x_test,y_test,beta):
    
    mean = np.zeros(3)  
    cov = np.eye(3)
    
    for i in range(x_test.shape[0]):
        count = 0
        while True:
            p = np.random.multivariate_normal(mean, cov,(48,48)) #sample formula
            neighbour = p + x_test[i]*255
            neighbour=neighbour.reshape(1,48,48,3)
            neighbour /= 255
            distance=np.linalg.norm(p) 
            distance /= 255
            #print(distance)
            if (distance < beta or distance == beta):
                output = lenet5.predict(neighbour,batch_size=128)[0]
                proba = softmax(output)
                pred = np.argmax(proba)
                label = y_test[i]
                if(pred == label): 
                    count+=1
                else:  #if there are some iamges inside beta cannot have the same label, beta will be decreased
                    beta = beta-0.0001
                    print("Beta is reduced by 0.0001, the new beta is",beta)  
                    i-=1
                    break
                    
            if(count<600):  
                continue
                
            if(count==600):  
                #local robustness formula satisfied
                break
                
    print("global robustness formula satisfied") 
    print("max safe distance:",beta)
    return beta
def softmax(x):
    e_x = np.exp(x-np.max(x))
    return e_x / e_x.sum()